In [2]:
import os
os.chdir("/content/drive/MyDrive/ug-project/src")
!pwd
%load_ext autoreload
%autoreload 2

/content/drive/MyDrive/ug-project/src


In [3]:
from datasets import ToyboxDataset, TOYBOX_MEAN, TOYBOX_STD
import numpy as np
import torchvision.transforms.v2 as v2
import torch.utils.data as torchdata
from torch.utils.data import DataLoader

### Set hyperparameters

In [5]:
learning_rate = (0.05, 0.01, 0.005, 0.001)
epochs = 10
batch_size = 256

### Define transforms

In [6]:
rng = np.random.default_rng(seed=5)

prob = 0.2
color_transforms = [v2.RandomApply([v2.ColorJitter(brightness=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(hue=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(saturation=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(contrast=0.2)], p=prob),
                    v2.RandomEqualize(p=prob),
                    v2.RandomPosterize(bits=4, p=prob),
                    v2.RandomAutocontrast(p=prob)
                    ]
transform = v2.Compose([v2.ToPILImage(),
                        v2.Resize((256, 256)),
                        v2.RandomResizedCrop(size=224, scale=(0.5, 1.0), interpolation=v2.InterpolationMode.BICUBIC),
                        v2.RandomOrder(color_transforms),
                        v2.RandomHorizontalFlip(),
                        v2.ToTensor(),
                        v2.Normalize(mean=TOYBOX_MEAN, std=TOYBOX_STD),
                        v2.RandomErasing(p=0.5)
                        ])
transform_test = v2.Compose([
    v2.ToPILImage(),
    v2.Resize(224),
    v2.ToTensor(),
    v2.Normalize(mean=TOYBOX_MEAN, std=TOYBOX_STD)
])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


### Load Toybox dataset

In [7]:
train_dataset = ToyboxDataset(rng=rng, train=True, hypertune=True, transform=transform, num_images_per_class=1500)
print(f"Dev set size: {len(train_dataset)}")

val_dataset = ToyboxDataset(rng=rng, train=False, hypertune=True, transform=transform_test, num_images_per_class=150)
print(f"Val set size: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Dev set size: 18000
Val set size: 13299


### Create ResNet18 model

In [8]:
import torch
import torch.nn as nn
from model import ResNet18Sup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

criterion = nn.CrossEntropyLoss()
steps = len(train_loader)

Using device: cuda


### Evaluate model before training

In [12]:
for lr in learning_rate:
    print(f"Learning rate: {lr}")
    model = ResNet18Sup(num_classes=12).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer=optimizer, start_factor=0.01, end_factor=1.0,
                                                          total_iters=2*steps)
    decay_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=(epochs - 2) * steps)
    combined_scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer=optimizer,
                                                                schedulers=[warmup_scheduler, decay_scheduler],
                                                                milestones=[2*steps+1])

    # Initialize metrics for ploting
    train_losses, train_correct = [], []
    val_losses, val_correct = [], []
    logs = []

    for epoch in range(0, epochs):

        ###################### Train model #########################
        model.train()
        for idx, images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            combined_scheduler.step()

        # Evaluate on training set
        model.eval()
        train_loss, train_corr, total_train = 0, 0, 0

        with torch.no_grad():
            for _, images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                y_pred = model(images)
                loss = criterion(y_pred, labels)

                train_loss += loss.item() * images.size(0)
                predicted = torch.max(y_pred.data, 1)[1]
                train_corr += (predicted == labels).sum().item()
                total_train += labels.size(0)

        avg_train_loss = train_loss / total_train
        train_accuracy = (train_corr / total_train) * 100

        ###################### Evaluate model ######################
        val_loss, val_corr, total_val = 0, 0, 0

        with torch.no_grad():
            for _, images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                y_val_pred = model(images)
                loss = criterion(y_val_pred, labels)

                val_loss += loss.item() * images.size(0)
                predicted = torch.max(y_val_pred, 1)[1]
                val_corr += (predicted == labels).sum().item()
                total_val += labels.size(0)


        avg_val_loss = val_loss / total_val
        val_accuracy = (val_corr / total_val) * 100

        # Save results of current epoch
        train_losses.append(avg_train_loss)
        train_correct.append(train_accuracy)
        val_losses.append(avg_val_loss)
        val_correct.append(val_accuracy)

        # Add epoch results to log file
        log_entry = (f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
        print(log_entry)
        logs.append(log_entry)

Learning rate: 0.05
Epoch 1/10, Train Loss: 5.4166, Train Acc: 16.60, Val Loss: 6.0707, Val Acc: 16.29%
Epoch 2/10, Train Loss: 2.1709, Train Acc: 31.83, Val Loss: 1.9719, Val Acc: 33.12%
Epoch 3/10, Train Loss: 1.9380, Train Acc: 36.93, Val Loss: 2.0398, Val Acc: 34.06%
Epoch 4/10, Train Loss: 1.5844, Train Acc: 45.59, Val Loss: 2.0590, Val Acc: 34.49%
Epoch 5/10, Train Loss: 1.4903, Train Acc: 48.66, Val Loss: 1.8320, Val Acc: 41.97%
Epoch 6/10, Train Loss: 1.3862, Train Acc: 54.08, Val Loss: 1.9392, Val Acc: 44.09%
Epoch 7/10, Train Loss: 1.1256, Train Acc: 61.86, Val Loss: 1.7209, Val Acc: 49.70%
Epoch 8/10, Train Loss: 0.8094, Train Acc: 72.14, Val Loss: 1.7260, Val Acc: 51.51%
Epoch 9/10, Train Loss: 0.6442, Train Acc: 77.77, Val Loss: 1.7111, Val Acc: 56.85%
Epoch 10/10, Train Loss: 0.5768, Train Acc: 79.90, Val Loss: 1.6161, Val Acc: 58.71%
Learning rate: 0.01
Epoch 1/10, Train Loss: 6.8012, Train Acc: 15.43, Val Loss: 9.3335, Val Acc: 14.63%
Epoch 2/10, Train Loss: 2.9056, Tra